In [ ]:
# import os, sys
# from pathlib import Path
# def get_project_root(marker="pyproject.toml"): # 현재 프로젝트의 경로를 가져오기 위해 사용용
#     cur = Path(__file__).resolve() if "__file__" in globals() else Path().resolve()
#     return next((p for p in [cur, *cur.parents] if (p / marker).exists()), cur)

# src_path = Path(os.getcwd()).resolve().parents[1]  
# sys.path.append(str(src_path))


In [15]:
import re

def parse_qa_pairs(qa_pair_text: str):
    """
    qa_pair 문자열에서 (question, answer) 쌍을 리스트로 추출.

    기대 형식 예:
    Q1: ...질문...
    A1: ...답변...

    Q2: ...질문...
    A2: ...답변...

    Q번호와 A번호가 맞는 구간을 정규식으로 잡아서 파싱.
    """
    qa_pair_text = qa_pair_text.strip()

    pattern = re.compile(
        r"Q(\d+):\s*(.+?)\nA\1:\s*(.+?)(?=(\nQ\d+:)|$)",
        re.DOTALL
    )

    pairs = []
    for m in pattern.finditer(qa_pair_text):
        q = m.group(2).strip()
        a = m.group(3).strip()

        # 답변에서 "출처: ..."는 임베딩엔 노이즈일 수 있으니 잘라낼지 선택
        a = re.sub(r"출처:.*", "", a, flags=re.DOTALL).strip()

        if q and a:
            pairs.append((q, a))
    return pairs


import os
import json
from glob import glob
from sentence_transformers import InputExample
from tqdm import tqdm


# BASE_DIR = str(get_project_root() / "src"/ "data" / "agent")
BASE_DIR = "data" / "agent"

# _pages 디렉토리 자동 탐색
pages_dirs = [
    os.path.join(BASE_DIR, d) for d in os.listdir(BASE_DIR)
    if d.endswith("_pages") and os.path.isdir(os.path.join(BASE_DIR, d))
]

qa_pairs = []

for folder in pages_dirs:
    jsonl_path = os.path.join(folder, "qa_dataset.jsonl")
    if not os.path.exists(jsonl_path):
        continue

    print(f"▶ 로딩: {jsonl_path}")
    with open(jsonl_path, "r", encoding="utf-8") as f:
        for line in f:
            if not line.strip():
                continue
            obj = json.loads(line)

            qa_text = obj.get("qa_pair", "")
            if not qa_text:
                continue

            # 🔥 여기서 한 줄 안의 Q1/A1, Q2/A2, ... 전부 쪼갬
            pairs = parse_qa_pairs(qa_text)

            qa_pairs.extend(pairs)

print(f"\n총 추출된 (Q, A) pair 개수: {len(qa_pairs)}")

▶ 로딩: /Users/seobi/PythonProjects/AxDeepScholar/src/data/agent/agent_2505.30656746v1.full_pages/qa_dataset.jsonl
▶ 로딩: /Users/seobi/PythonProjects/AxDeepScholar/src/data/agent/agent_2411.00114v1_pages/qa_dataset.jsonl
▶ 로딩: /Users/seobi/PythonProjects/AxDeepScholar/src/data/agent/agent_2502.14143v1_pages/qa_dataset.jsonl
▶ 로딩: /Users/seobi/PythonProjects/AxDeepScholar/src/data/agent/agent_2504.16736v3_pages/qa_dataset.jsonl
▶ 로딩: /Users/seobi/PythonProjects/AxDeepScholar/src/data/agent/agent_2501.00083v1_pages/qa_dataset.jsonl
▶ 로딩: /Users/seobi/PythonProjects/AxDeepScholar/src/data/agent/agent_2506.19676v3_pages/qa_dataset.jsonl
▶ 로딩: /Users/seobi/PythonProjects/AxDeepScholar/src/data/agent/agent_2408.07199v1_pages/qa_dataset.jsonl
▶ 로딩: /Users/seobi/PythonProjects/AxDeepScholar/src/data/agent/agent_2503.10970v1_pages/qa_dataset.jsonl
▶ 로딩: /Users/seobi/PythonProjects/AxDeepScholar/src/data/agent/agent_2411.15594v5_pages/qa_dataset.jsonl
▶ 로딩: /Users/seobi/PythonProjects/AxDeepScholar

In [22]:
qa_pairs[0]

('멀티 에이전트 시스템에서 에이전트들의 문명 진보를 위한 벤치마크가 부재한 이유는 무엇이며, 이로 인한 연구의 한계는 무엇인가요?',
 '멀티 에이전트 시스템에서 문명 진보를 위한 벤치마크가 부재한 이유는 기존의 벤치마크들이 웹 검색, 코딩, 검색 및 질의, 추론과 같은 다양한 도메인에서의 자율적 에이전트 성능에 집중되어 있기 때문입니다. 이로 인한 연구의 한계는 에이전트들이 사회적, 문화적, 기술적 진보를 이루는 데 필요한 복잡한 상호작용과 장기적인 목표 달성을 측정하는 데 어려움을 겪게 된다는 점입니다. 이는 멀티 에이전트 그룹이 협력하여 문명을 발전시키는 과정을 평가하는 데 필요한 새로운 벤치마크의 필요성을 시사합니다.')

In [17]:
import random

# qa_pairs = [(question, answer), ...]
random.shuffle(qa_pairs)

split = int(len(qa_pairs) * 0.9)
train_pairs = qa_pairs[:split]
val_pairs = qa_pairs[split:]

print("Train pairs:", len(train_pairs))
print("Val pairs:", len(val_pairs))


Train pairs: 2059
Val pairs: 229


In [18]:
from sentence_transformers import InputExample

train_examples = [
    InputExample(texts=[q, a]) for q, a in train_pairs
]

print("첫 샘플:", train_examples[0].texts)

첫 샘플: ['멀티 에이전트 시스템에서 에이전트들의 문명 진보를 위한 벤치마크가 부재한 이유는 무엇이며, 이로 인한 연구의 한계는 무엇인가요?', '멀티 에이전트 시스템에서 문명 진보를 위한 벤치마크가 부재한 이유는 기존의 벤치마크들이 웹 검색, 코딩, 검색 및 질의, 추론과 같은 다양한 도메인에서의 자율적 에이전트 성능에 집중되어 있기 때문입니다. 이로 인한 연구의 한계는 에이전트들이 사회적, 문화적, 기술적 진보를 이루는 데 필요한 복잡한 상호작용과 장기적인 목표 달성을 측정하는 데 어려움을 겪게 된다는 점입니다. 이는 멀티 에이전트 그룹이 협력하여 문명을 발전시키는 과정을 평가하는 데 필요한 새로운 벤치마크의 필요성을 시사합니다.']


In [19]:
from torch.utils.data import DataLoader

BATCH_SIZE = 16   # GPU 상황 보고 조절
train_dataloader = DataLoader(
    train_examples,
    shuffle=True,
    batch_size=BATCH_SIZE
)

In [20]:
from sentence_transformers import SentenceTransformer, losses

model_id = "BAAI/bge-m3"
model = SentenceTransformer(model_id)

loss = losses.MultipleNegativesRankingLoss(model)

In [21]:
from sentence_transformers.evaluation import InformationRetrievalEvaluator

# val_pairs = [(q, a), ...]
val_dataset = {
    "queries": {},
    "corpus": {},
    "relevant_docs": {}
}

for i, (q, a) in enumerate(val_pairs):
    qid = f"q{i}"
    did = f"d{i}"
    val_dataset["queries"][qid] = q
    val_dataset["corpus"][did] = a
    val_dataset["relevant_docs"][qid] = {did}  # 이 질문의 정답 문서는 자기 answer 하나

corpus = val_dataset["corpus"]
queries = val_dataset["queries"]
relevant_docs = val_dataset["relevant_docs"]

evaluator = InformationRetrievalEvaluator(
    queries,
    corpus,
    relevant_docs
)

print("질문 개수:", len(queries))
print("코퍼스 문서 개수:", len(corpus))

질문 개수: 229
코퍼스 문서 개수: 229


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

EPOCHS = 2
warmup_steps = int(len(train_dataloader) * EPOCHS * 0.1)

model.fit(
    train_objectives=[(train_dataloader, loss)],
    epochs=EPOCHS,
    warmup_steps=warmup_steps,
    output_path="bge3_finetuned",
    show_progress_bar=True
)

In [ ]:
import pandas as pd
def evaluate_st(dataset, model_id, name, evaluator):
    """
    SentenceTransformer 모델의 검색 성능을 평가하는 함수
    """
    os.makedirs('results', exist_ok=True)

    model = SentenceTransformer(model_id)

    result = evaluator(model)

    result_df = pd.DataFrame([result]) if isinstance(result, dict) else result
    output_path = f'results/Information-Retrieval_evaluation_{name}_results.csv'
    result_df.to_csv(output_path, index=False)

    return result

# 파인튜닝 모델 평가
finetuned_model_path = "exp_finetune"
evaluate_st(dataset=val_dataset, model_id=finetuned_model_path, name="finetuned", evaluator=evaluator)
print("파인튜닝 모델 평가 완료")

# 원본 bge-m3 평가
original_model_path = "BAAI/bge-m3"
evaluate_st(dataset=val_dataset, model_id=original_model_path, name="original", evaluator=evaluator)
print("원본 모델 평가 완료")

# 결과 비교
df_st_original = pd.read_csv('results/Information-Retrieval_evaluation_original_results.csv')
df_st_finetuned = pd.read_csv('results/Information-Retrieval_evaluation_finetuned_results.csv')

df_st_original['model'] = 'bge-m3'
df_st_finetuned['model'] = 'fine_tuned'
df_st_all = pd.concat([df_st_original, df_st_finetuned])
df_st_all = df_st_all.set_index('model')

print("\n모델 성능 비교:")
df_st_all